In [1]:
import os
import pandas as pd
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import numpy as np
import numpy.linalg as lin

# CASBAH QSO COORDS

## Step 1: Read in casbah qso ra and dec coordinates

In [ ]:
#ra, dec, and redshift (ra and dec in degrees)
qso_coordz = {
    'PHL1377': [38.78077, -4.03491, 1.437],
    'FBQSJ0751+2919': [117.80128, 29.32730, 0.916],
    'PG1206+459': [182.24172, 45.67652, 1.165],
    'PG1407+265': [212.34963, 26.30587, 0.94],
    'LBQS1435-0134': [219.45118, -1.78633, 1.311],
    'PG1522+101': [231.10231, 9.97494, 1.328],
    'PG1630+377': [248.00462, 37.63055, 1.479]
}

In [10]:
qso_coords= pd.DataFrame(qso_coordz).transpose()
qso_coords= qso_coords.rename(columns={0: "RA", 1: "DEC", 2:"redshift"})
#qso_coords.rename(index={0: "x", 1: "y", 2: "z"})

In [61]:
qso_coords

,RA,DEC,redshift
PHL1377,38.78077,-4.03491,1.437
FBQSJ0751+2919,117.80128,29.32730,0.916
PG1206+459,182.24172,45.67652,1.165
PG1407+265,212.34963,26.30587,0.940
LBQS1435-0134,219.45118,-1.78633,1.311
PG1522+101,231.10231,9.97494,1.328
PG1630+377,248.00462,37.63055,1.479


## Step 2: Read in all the galaxy files:
- convert these all to 3 giant combined data frames across the 3 data types

# add all qso coords here to the casbah dataframe

In [58]:
import pandas as pd 
import os 
import glob 

# create paths to folders
path_cwd = os.getcwd() 
path_casbah= path_cwd+'/casbah/'
path_initial= path_cwd+'/cgm2_initial/'
path_leftover= path_cwd+'/cgm2_leftover/'

#collect all files from each folder
csv_files_casbah = glob.glob(os.path.join(path_casbah, "*.csv")) 
csv_files_initial = glob.glob(os.path.join(path_initial, "*.csv")) 
csv_files_leftover = glob.glob(os.path.join(path_leftover, "*.csv")) 

# loop over the list of csv files, combine them into large DF
# start with an empty dataframe (instead of an empty list)
casbah= pd.DataFrame()
for f in csv_files_casbah: 
    # read the csv file 
    df = pd.read_csv(f)
    # add the name of the corresponding QSO to the proper column
    df['QSO']= str(f)[40:].split('_')[0]
    for index, row in qso_coords.iterrows():
        #print(str(f)[40:].split('_')[0][0:6])
        if str(f)[40:].split('_')[0][0:6] == str(index)[0:6]:
            df['QSO_RA']= row['RA']
            df['QSO_DEC']=row['DEC']
    # Both of these methods work but concat is the newest/ proper method...
    #casbah = casbah.append(df)
    casbah = pd.concat([casbah, df])
    
cgm_initial= pd.DataFrame()
for f in csv_files_initial: 
    # read the csv file 
    df = pd.read_csv(f) 
    # add QSO name
    df['QSO']= str(f)[46:].split('_')[0]
    #cgm_initial= cgm_initial.append(df)
    cgm_initial = pd.concat([cgm_initial, df])
    
cgm_leftover= pd.DataFrame()
for f in csv_files_leftover: 
    # read the csv file 
    df = pd.read_csv(f) 
    df['QSO']= str(f)[47:].split('_')[0]
    #cgm_leftover= cgm_leftover.append(df)
    cgm_leftover = pd.concat([cgm_leftover, df])

LBQS14
LBQS14
PG1407
PG1407
FBQSJ0
FBQSJ0
PHL137
PHL137
PG1630
PG1630
PG1522
PG1522
PG1206
PG1206


In [3]:
cgm_leftover['QSO'].unique()

array(['J1233-0031', 'J1059+2517', 'J0935+0204', 'J2345-0059',
       'J1555+3628', 'J0809+4619', 'J1342-0053', 'J1133+0327',
       'J1134+2555', 'J1059+1441', 'J1016+4706', 'J1419+4207',
       'J1553+3548', 'J1001+5944', 'J0843+4117', 'J1241+5721',
       'J1437+5045', 'J1112+3539', 'J0943+0531', 'J1022+0132',
       'J0914+2823'], dtype=object)

In [4]:
casbah['QSO'].unique()

array(['LBQS1435-0134', 'PG1407+265', 'FBQSJ0751+2919', 'PHL1377',
       'PG1630+377', 'PG1522+101', 'PG1206+459'], dtype=object)

In [5]:
casbah.columns, cgm_initial.columns, cgm_leftover.columns

(Index(['gal_num', 'redshift', 'xcoord', 'xcoord_error', 'ycoord',
        'ycoord_error', 'int_mag', 'int_mag_error', 'half_light_rad',
        'half_light_rad_error', 'sersic_index', 'sersic_index_error',
        'axis_ratio', 'axis_ratio_error', 'position_angle',
        'position_angle_error', 'reduced_chi_squared', 'inclination_angle',
        'is_good_fit', 'shows_internal_struct', 'overlaps_w_object',
        'is_big_galaxy', 'edited', 'double_model', 'QSO'],
       dtype='object'),
 Index(['gal_num', 'redshift', 'xcoord', 'xcoord_error', 'ycoord',
        'ycoord_error', 'int_mag', 'int_mag_error', 'half_light_rad',
        'half_light_rad_error', 'sersic_index', 'sersic_index_error',
        'axis_ratio', 'axis_ratio_error', 'position_angle',
        'position_angle_error', 'reduced_chi_squared', 'azimuthal_angle',
        'inclination_angle', 'is_good_fit', 'shows_internal_struct',
        'overlaps_w_object', 'is_big_galaxy', 'edited', 'double_model', 'QSO',
        'Unnamed

## For some reason there are a few NaN columns...
- deleting those here

In [6]:
bad_columns= ['Unnamed: 0', 'Unnamed: 0.2', 'Unnamed: 0.1']
#del cgm_initial[]
for bad in bad_columns:
    del cgm_initial[bad]

## Fixed!

In [8]:
print(len(casbah.columns)), print(len(cgm_initial.columns)), print(len(cgm_leftover.columns))

25
26
26


(None, None, None)

# First issue:
- the casbah data doesnt have azimuthal data included

## Step 2: find azimuthal angles for casbah...
- thomas code from github + a lot of modifications...

## Need to add a column for the name of the qso in casbah

## Edit code here

In [95]:
path_images= path_casbah+'images/'
# major indexing issues with casbah... this fixes a lot
#casbah= casbah.reset_index()
phi = []
for image in glob.glob(os.path.join(path_images, "*")):
    hdulist = fits.open(image, memmap=True)
    hdu = hdulist[0]
    wcs = WCS(hdu.header)
    hdulist.close()
    #print(image)
    
    # make sure that the correct qso data goes to the right galaxy
    for index, row in casbah.iterrows():
        if image[47:53]== row['QSO'][0:6]:
            #print("TRUE")
            
            # get the ra and dec for the qso
            qso_ra = row['QSO_RA']
            qso_dec = row['QSO_DEC']
            #print(qso_ra)

            # Converts RA/DEC of the QSO to image x/y
            coord = SkyCoord(qso_ra, qso_dec, unit='deg')
            object_pix = wcs.world_to_pixel(coord)
            qso_pix_coordx = object_pix[0]
            qso_pix_coordy = object_pix[1]

            # Finds the changes in pixel x and y coordinates from each galaxy to the QSO
            delta_x = qso_pix_coordx - row['xcoord']
            delta_y = qso_pix_coordy - row['ycoord']
            #print(delta_x)
            
            # Calculates the azimuthal angle phi for each galaxy
            #phi = []
            delta_coords= (delta_x,delta_y)
            
            # for i in np.arange(0,len(delta_x),1):
            #if casbah['position_angle'][i] != 'oob':
            # Create vector a which is the vector pointing from galaxy to QSO
            #a = [delta_x[i], delta_y[i]]
            a= delta_coords

            # Create vectors b1 and b2 which are vectors pointing along galaxy's semi-minor axis
            # b1 and b2 are anti-parallel

            # finds the position angle of each galaxy in casbah table and converts it to radians
            position_angle = row['position_angle'] * (np.pi / 180)
            #print("POSANG",position_angle)
            #break

            b1 = (np.cos(position_angle), np.sin(position_angle))
            b2 = (-np.cos(position_angle), -np.sin(position_angle))
            #print("B1", b1)
            #print("B2",b2)

            # now sort them by QSO and finish the process using [QSO] columns i added to casbah

            # print(np.dot(a, b1))
            # Uses the relationship between the cross product of two vectors and cosine to find the angle between the two vectors
            phi_prime1 = np.arccos(np.dot(a, b1) / (lin.norm(a) * lin.norm(b1))) * (180 / np.pi)
            phi_prime2 = np.arccos(np.dot(a, b2) / (lin.norm(a) * lin.norm(b2))) * (180 / np.pi)

            # Takes the lesser of the two angles since azimuthal angle can go from 0 to +90 degrees
            if (phi_prime1 <= phi_prime2):
                phi.append(phi_prime1)
            elif (phi_prime1 > phi_prime2):
                phi.append(phi_prime2)
            # we only need it to do this once
        # we only need it to do this once
casbah['azimuthal_angle']= phi
phi

[55.258767751207294,
 56.23603143754632,
 36.19550143012011,
 4.259117933359723,
 63.86299374702369,
 9.289515374340374,
 21.242420188377736,
 15.690347943690234,
 87.7508157235413,
 63.328079226233456,
 79.99944246847119,
 10.763005575991807,
 54.89210893776743,
 1.528984512084501,
 46.10580157635807,
 82.05157451687018,
 45.42568528696985,
 58.78342054054217,
 83.03669417786969,
 26.064283955592952,
 56.17164020530202,
 19.280661432016046,
 34.24781081910633,
 22.76284642386056,
 54.43889705569868,
 50.86319350511441,
 27.87547707675121,
 34.764959007719575,
 2.649446054095337,
 74.40523891247834,
 74.95924848730755,
 51.676879387704865,
 75.17578599165668,
 87.55978024786728,
 31.433351934958086,
 9.392991373616004,
 44.43410362210071,
 89.17458013996337,
 43.15511529267414,
 16.19753879076546,
 12.875769703447576,
 31.015141248831238,
 70.06054548018682,
 14.293493483379228,
 63.46192408542024,
 6.100389557460833,
 53.671589230645004,
 61.357327978868376,
 61.75156931576699,
 24.19

## original code

In [96]:
# stored from it already working...

In [97]:
casbah

,gal_num,redshift,xcoord,xcoord_error,ycoord,ycoord_error,int_mag,int_mag_error,half_light_rad,half_light_rad_error,...,is_good_fit,shows_internal_struct,overlaps_w_object,is_big_galaxy,edited,double_model,QSO,QSO_RA,QSO_DEC,azimuthal_angle
0,0,0.184300,1640.88,0.01,1958.44,0.01,-23.00,0.01,1.55,0.02,...,False,False,False,False,False,False,LBQS1435-0134,219.45118,-1.78633,55.258768
1,1,0.412960,2654.21,0.26,2575.02,0.42,-19.39,0.22,2.18,0.56,...,True,False,False,False,False,False,LBQS1435-0134,219.45118,-1.78633,56.236031
2,2,0.508760,2637.12,0.27,2009.50,0.17,-20.65,0.12,3.67,0.29,...,True,False,False,True,False,False,LBQS1435-0134,219.45118,-1.78633,36.195501
3,3,0.546230,2598.76,0.54,2490.22,0.97,-22.81,0.15,13.16,1.66,...,True,False,False,True,False,False,LBQS1435-0134,219.45118,-1.78633,4.259118
4,4,0.611740,2240.16,0.01,2756.29,0.07,-19.61,0.06,4.12,0.03,...,True,False,False,True,False,False,LBQS1435-0134,219.45118,-1.78633,63.862994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,1,0.206728,2120.92,0.02,2242.17,0.01,-25.81,0.01,8.68,0.12,...,True,False,False,True,False,False,PG1206+459,182.24172,45.67652,3.253816
2,2,0.214700,2345.13,0.11,2461.21,0.12,-25.91,1.01,46.46,40.39,...,True,False,False,True,False,False,PG1206+459,182.24172,45.67652,63.870810
3,3,0.415000,1775.95,1.31,2189.45,0.99,-21.02,0.69,2.79,1.41,...,False,False,False,False,False,False,PG1206+459,182.24172,45.67652,24.265114
4,5,0.427500,2106.28,0.01,3251.47,0.00,-27.43,0.00,6.92,0.04,...,True,False,False,True,False,False,PG1206+459,182.24172,45.67652,9.061365


In [98]:
casbah.columns

Index(['gal_num', 'redshift', 'xcoord', 'xcoord_error', 'ycoord',
       'ycoord_error', 'int_mag', 'int_mag_error', 'half_light_rad',
       'half_light_rad_error', 'sersic_index', 'sersic_index_error',
       'axis_ratio', 'axis_ratio_error', 'position_angle',
       'position_angle_error', 'reduced_chi_squared', 'inclination_angle',
       'is_good_fit', 'shows_internal_struct', 'overlaps_w_object',
       'is_big_galaxy', 'edited', 'double_model', 'QSO', 'QSO_RA', 'QSO_DEC',
       'azimuthal_angle'],
      dtype='object')

## Removing the QSO_RA and QSO_DEC columns from the code to add the frames together
- all should have same # of columns

In [104]:
print(len(casbah.columns)), print(len(cgm_initial.columns)), print(len(cgm_leftover.columns))

28
26
26


(None, None, None)

In [111]:
# del casbah['QSO_RA']
# del casbah['QSO_DEC']

In [112]:
casbah.columns

Index(['gal_num', 'redshift', 'xcoord', 'xcoord_error', 'ycoord',
       'ycoord_error', 'int_mag', 'int_mag_error', 'half_light_rad',
       'half_light_rad_error', 'sersic_index', 'sersic_index_error',
       'axis_ratio', 'axis_ratio_error', 'position_angle',
       'position_angle_error', 'reduced_chi_squared', 'inclination_angle',
       'is_good_fit', 'shows_internal_struct', 'overlaps_w_object',
       'is_big_galaxy', 'edited', 'double_model', 'QSO', 'azimuthal_angle'],
      dtype='object')

In [102]:
del cgm_initial['Unnamed: 0']
del cgm_initial['Unnamed: 0.2']
del cgm_initial['Unnamed: 0.1']

In [114]:
cgm_leftover.columns

Index(['gal_num', 'redshift', 'xcoord', 'xcoord_error', 'ycoord',
       'ycoord_error', 'int_mag', 'int_mag_error', 'half_light_rad',
       'half_light_rad_error', 'sersic_index', 'sersic_index_error',
       'axis_ratio', 'axis_ratio_error', 'position_angle',
       'position_angle_error', 'reduced_chi_squared', 'inclination_angle',
       'is_good_fit', 'shows_internal_struct', 'overlaps_w_object',
       'is_big_galaxy', 'edited', 'double_model', 'QSO', 'azimuthal_angle'],
      dtype='object')

## making sure azimuthal angles/column names are all in the same spot...

In [109]:
cgm_initial = cgm_initial[['gal_num', 'redshift', 'xcoord', 'xcoord_error', 'ycoord',
       'ycoord_error', 'int_mag', 'int_mag_error', 'half_light_rad',
       'half_light_rad_error', 'sersic_index', 'sersic_index_error',
       'axis_ratio', 'axis_ratio_error', 'position_angle',
       'position_angle_error', 'reduced_chi_squared',
       'inclination_angle', 'is_good_fit', 'shows_internal_struct',
       'overlaps_w_object', 'is_big_galaxy', 'edited', 'double_model', 'QSO', 'azimuthal_angle']]
cgm_initial

,gal_num,redshift,xcoord,xcoord_error,ycoord,ycoord_error,int_mag,int_mag_error,half_light_rad,half_light_rad_error,...,reduced_chi_squared,inclination_angle,is_good_fit,shows_internal_struct,overlaps_w_object,is_big_galaxy,edited,double_model,QSO,azimuthal_angle
0,1728,0.015210,1167.27,0.05,679.56,0.04,16.25,0.01,4.81,0.11,...,1.110,60.000000,True,False,True,True,True,False,J1342-0053,76.049434
1,1729,0.071141,1663.52,0.00,1061.76,0.00,12.61,0.00,6.22,0.03,...,0.875,45.572996,True,True,False,True,True,False,J1342-0053,48.017481
2,1731,0.114920,818.03,0.04,1547.88,0.01,14.50,0.00,10.91,0.05,...,1.138,76.702928,True,True,False,True,False,False,J1342-0053,87.433226
3,1734,0.178165,1026.11,0.11,1258.15,0.05,16.12,0.02,7.01,0.22,...,1.144,77.877648,True,False,False,True,False,False,J1342-0053,65.266082
4,1742,0.310258,1709.92,0.08,637.75,0.09,16.95,0.04,2.59,0.22,...,1.288,66.421822,True,False,False,False,False,False,J1342-0053,88.692151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,852,0.400820,4116.03,0.01,2641.09,0.01,13.27,0.01,9.90,0.14,...,0.811,42.268584,True,True,False,True,True,False,J1016+4706,60.797316
14,859,0.472508,2659.85,0.09,3034.45,0.10,15.43,0.01,8.79,0.17,...,1.044,34.915206,True,False,False,True,False,False,J1016+4706,40.420450
15,860,0.472939,2737.14,0.08,3234.68,0.10,16.81,0.04,2.89,3.01,...,1.102,67.666317,True,False,False,False,False,False,J1016+4706,48.443678
16,864,0.483531,3988.99,0.11,1844.67,0.13,16.58,0.07,5.55,0.50,...,1.066,37.814489,True,False,False,True,False,False,J1016+4706,61.100673


In [110]:
cgm_leftover = cgm_leftover[['gal_num', 'redshift', 'xcoord', 'xcoord_error', 'ycoord',
       'ycoord_error', 'int_mag', 'int_mag_error', 'half_light_rad',
       'half_light_rad_error', 'sersic_index', 'sersic_index_error',
       'axis_ratio', 'axis_ratio_error', 'position_angle',
       'position_angle_error', 'reduced_chi_squared',
       'inclination_angle', 'is_good_fit', 'shows_internal_struct',
       'overlaps_w_object', 'is_big_galaxy', 'edited', 'double_model', 'QSO', 'azimuthal_angle']]
cgm_leftover

,gal_num,redshift,xcoord,xcoord_error,ycoord,ycoord_error,int_mag,int_mag_error,half_light_rad,half_light_rad_error,...,reduced_chi_squared,inclination_angle,is_good_fit,shows_internal_struct,overlaps_w_object,is_big_galaxy,edited,double_model,QSO,azimuthal_angle
0,1612,0.193030,2087.61,0.06,1970.73,0.09,12.88,0.01,15.20,0.11,...,4.705,52.410497,False,True,True,True,False,False,J1233-0031,37.579913
1,1624,0.318496,1625.47,1.75,1236.02,1.53,15.64,0.37,4.27,20.69,...,861.875,32.859880,True,False,False,True,False,False,J1233-0031,36.933422
0,1130,0.020901,1355.96,0.03,2227.53,0.03,8.82,0.19,207.26,45.42,...,0.391,39.646111,False,True,False,True,False,False,J1059+2517,53.568683
0,497,0.092302,5029.98,0.10,926.77,0.08,15.60,0.04,8.43,0.45,...,1.049,46.369891,True,False,False,True,False,False,J0935+0204,85.870873
1,502,0.125298,2020.51,0.04,2578.34,0.02,13.71,0.01,11.55,0.09,...,0.845,57.316361,True,False,False,True,False,False,J0935+0204,76.114832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,627,0.228386,1389.90,0.02,1426.19,0.01,13.22,0.00,8.52,0.06,...,1.319,50.208181,False,True,False,True,False,False,J0943+0531,58.229214
3,644,0.352952,1635.30,0.04,1221.06,0.04,15.84,0.01,2.81,0.08,...,1.101,41.409622,True,False,False,False,False,False,J0943+0531,85.468753
0,949,0.074366,2743.58,0.12,2583.77,0.11,15.45,0.03,12.01,0.38,...,0.963,57.316361,False,False,True,True,True,False,J1022+0132,60.893920
0,366,0.226509,4168.02,0.02,976.13,0.05,11.84,0.02,32.56,0.57,...,1.277,63.256316,True,False,False,True,False,False,J0914+2823,23.596928


# Combining data frames from each subsection

In [115]:
combined_gals= pd.DataFrame()
frames= casbah,cgm_initial, cgm_leftover
for frame in frames:
    df= pd.concat([combined_gals, frame])
    combined_gals= df

In [116]:
combined_gals.to_csv('casbah_all_cgm_combined_ally.csv')

In [117]:
casbah['azimuthal_angle']

0    55.258768
1    56.236031
2    36.195501
3     4.259118
4    63.862994
       ...    
1     3.253816
2    63.870810
3    24.265114
4     9.061365
5    31.612073
Name: azimuthal_angle, Length: 92, dtype: float64